In [ ]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel'
model_name = "gpt-3.5-turbo"
label_column = 'sentence_class'
example_selection_label_column = 'sentence_check_class'
text_column = 'tweet'
explanation_column = 'sentence_check_explanation'
reference_column = 'sentence_check_reference'
few_shot_num = 8
few_shot_selection = "label_diversity_similarity"
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
random_shuffle_examples = True
random_shuffle_examples_seed = 1

In [ ]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
import sys

In [ ]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [ ]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [ ]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [ ]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [ ]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name.split('/')[1]}"

In [ ]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_check_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [ ]:
test_data = pd.read_csv(data_path)

In [ ]:
if reference_column.startswith('dummy_'):
    test_data[reference_column] = test_data[reference_column[len('dummy_'):]]

In [ ]:
config = {
    "task_name": task_name,
    "task_type": "question_answering",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "explanation_column": explanation_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ","
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": ("You are an expert at identifying foodborne illness incident information. For the given text, "
                            "your task is to evaluate the text to determine if it describes a potential foodborne illness event. "
                            "Another model has extracted some entities that are related to foodborne illness incident, you can take it as a reference. But the finding might be incorrect. "
                            "Use the following examples as a guide for your predictions and format your responses similarly."
                  ),
        "output_guidelines": ('''Your answer will consist of an explanation, followed by the correct answer ("Yes" or "No").'''
                              '''Please answer with "Yes" if it describes a potential foodborne illness event, otherwise answer with "No".'''
                              '''The last line of the response should always be JSON format with one key: {"label": "the correct answer"}.\n'''),
        "labels": [
            "Yes",
            "No"
        ],
        "example_selection_labels":[
            "YesYes",
            "NoYes",
            "YesNo",
            "NoNo",
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Context: {{{text_column}}}\nFinding: {{{reference_column}}}\nAnswer: Let's think step by step.\n{{{explanation_column}}}\n{{{label_column}}}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [ ]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-28 05:48:20 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
ds = AutolabelDataset(test_data[[text_column, reference_column, label_column]], config=config)
agent.plan(ds)

In [ ]:
# now, do the actual labeling
ds = agent.run(ds)

In [ ]:
metrics = ds.eval()

In [ ]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))